In [15]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Utokyo/lecture/2024DLBasic/dl_lecture_competition_pub

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Utokyo/lecture/2024DLBasic/dl_lecture_competition_pub


### ライブラリのインストール

ここは、Colabにあるやつはやらなくてもいいかも

要検討ポイント

In [16]:
%pip install -r requirements.txt

In [17]:
%cd /content/drive/MyDrive/Utokyo/lecture/2024DLBasic/dl_lecture_competition_pub
import os, sys
import numpy as np
import torch
import torch.nn.functional as F
from torchmetrics import Accuracy
import hydra
from omegaconf import DictConfig
from termcolor import cprint
from tqdm import tqdm
import random
from typing import Tuple

/content/drive/MyDrive/Utokyo/lecture/2024DLBasic/dl_lecture_competition_pub


In [18]:
# set_seedの定義
def set_seed(seed: int = 0) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [29]:
args = {
    "batch_size": 128,
    "epochs": 30,
    "lr": 0.001,

    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "num_workers": 4,
    "seed": 1234,
    "data_dir": 'data',

    # Evaluation
    "model_path": '',
}

logdir = './output'  # Specify your output directory
os.makedirs(logdir, exist_ok=True)

### Dataloader

大体これに5分くらいかかる

In [20]:
# Datasetのクラスの定義、ここで前処理とかをしばく？？
class ThingsMEGDataset(torch.utils.data.Dataset):
    def __init__(self, split: str, data_dir: str = "data") -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.num_classes = 1854

        self.X = torch.load(os.path.join(data_dir, f"{split}_X.pt"))
        self.subject_idxs = torch.load(os.path.join(data_dir, f"{split}_subject_idxs.pt"))

        if split in ["train", "val"]:
            self.y = torch.load(os.path.join(data_dir, f"{split}_y.pt"))
            assert len(torch.unique(self.y)) == self.num_classes, "Number of classes do not match."

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

In [21]:
loader_args = {"batch_size": args['batch_size'], "num_workers": args['num_workers']}

train_set = ThingsMEGDataset("train", args['data_dir'])
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
val_set = ThingsMEGDataset("val", args['data_dir'])
val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)
test_set = ThingsMEGDataset("test", args['data_dir'])
test_loader = torch.utils.data.DataLoader(
    test_set, shuffle=False, batch_size=args['batch_size'], num_workers=args['num_workers']
)

### Model

In [22]:

import torch.nn as nn
import torch.nn.functional as F
from einops.layers.torch import Rearrange


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=3, padding="same")
        self.bn1 = nn.BatchNorm1d(channels)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=3, padding="same")
        self.bn2 = nn.BatchNorm1d(channels)

    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual  # Residual connection
        return F.relu(x)

class ImprovedConvClassifier(nn.Module):
    def __init__(
        self,
        num_classes: int,
        seq_len: int,
        in_channels: int,
        hid_dim: int = 512,  # 隠れ層の次元数を増やす
        p_drop: float = 0.4,
    ) -> None:
        super().__init__()

        self.blocks = nn.Sequential(
            ConvBlock(in_channels, hid_dim),
            ResidualBlock(hid_dim),  # Residual Blockを追加
            ConvBlock(hid_dim, hid_dim * 2),  # チャンネル数を増やす
            ResidualBlock(hid_dim * 2),
            ConvBlock(hid_dim * 2, hid_dim * 2),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            Rearrange("b d 1 -> b d"),
            nn.Dropout(p_drop),  # 全結合層の前にDropoutを追加
            nn.Linear(hid_dim * 2, num_classes),  # チャンネル数に合わせて変更
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.blocks(X)
        return self.head(X)


class ConvBlock(nn.Module):
    def __init__(
        self,
        in_dim,
        out_dim,
        kernel_size: int = 3,
        p_drop: float = 0.4,
    ) -> None:
        super().__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim

        self.conv0 = nn.Conv1d(in_dim, out_dim, kernel_size, padding="same")
        self.conv1 = nn.Conv1d(out_dim, out_dim, kernel_size, padding="same")
        # self.conv2 = nn.Conv1d(out_dim, out_dim, kernel_size) # , padding="same")

        self.batchnorm0 = nn.BatchNorm1d(num_features=out_dim)
        self.batchnorm1 = nn.BatchNorm1d(num_features=out_dim)

        self.dropout = nn.Dropout(p_drop)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        if self.in_dim == self.out_dim:
            X = self.conv0(X) + X  # skip connection
        else:
            X = self.conv0(X)

        X = F.gelu(self.batchnorm0(X))

        X = self.conv1(X) + X  # skip connection
        X = F.gelu(self.batchnorm1(X))

        # X = self.conv2(X)
        # X = F.glu(X, dim=-2)

        return self.dropout(X)


In [27]:
model = ImprovedConvClassifier(
    train_set.num_classes, train_set.seq_len, train_set.num_channels
).to(args['device'])

model.load_state_dict(torch.load(os.path.join(logdir, "model_best.pt"), map_location=args['device']))

<All keys matched successfully>

### Optimizer

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

### Training

In [30]:
max_val_acc = 0
accuracy = Accuracy(
    task="multiclass", num_classes=train_set.num_classes, top_k=10
).to(args['device'])

for epoch in range(args['epochs']):
    print(f"Epoch {epoch+1}/{args['epochs']}")

    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    model.train()
    for X, y, subject_idxs in tqdm(train_loader, desc="Train"):
        X, y = X.to(args['device']), y.to(args['device'])

        y_pred = model(X)

        loss = F.cross_entropy(y_pred, y)
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = accuracy(y_pred, y)
        train_acc.append(acc.item())

    model.eval()
    for X, y, subject_idxs in tqdm(val_loader, desc="Validation"):
        X, y = X.to(args['device']), y.to(args['device'])

        with torch.no_grad():
            y_pred = model(X)

        val_loss.append(F.cross_entropy(y_pred, y).item())
        val_acc.append(accuracy(y_pred, y).item())

    print(f"Epoch {epoch+1}/{args['epochs']} | train loss: {np.mean(train_loss):.3f} | train acc: {np.mean(train_acc):.3f} | val loss: {np.mean(val_loss):.3f} | val acc: {np.mean(val_acc):.3f}")
    torch.save(model.state_dict(), os.path.join(logdir, "model_last.pt"))
    # if args['use_wandb']:
    #     wandb.log({"train_loss": np.mean(train_loss), "train_acc": np.mean(train_acc), "val_loss": np.mean(val_loss), "val_acc": np.mean(val_acc)})

    if np.mean(val_acc) > max_val_acc:
        cprint("New best.", "cyan")
        torch.save(model.state_dict(), os.path.join(logdir, "model_best.pt"))
        max_val_acc = np.mean(val_acc)

Epoch 1/5


Validation: 100%|██████████| 129/129 [00:54<00:00,  2.36it/s]


Epoch 1/5 | train loss: 7.361 | train acc: 0.020 | val loss: 7.470 | val acc: 0.018
New best.
Epoch 2/5


Validation: 100%|██████████| 129/129 [00:54<00:00,  2.36it/s]


Epoch 2/5 | train loss: 7.334 | train acc: 0.023 | val loss: 7.485 | val acc: 0.018
Epoch 3/5


...

Epoch 3/5 | train loss: 7.316 | train acc: 0.023 | val loss: 7.500 | val acc: 0.017
Epoch 4/5


Validation: 100%|██████████| 129/129 [00:54<00:00,  2.37it/s]


Epoch 4/5 | train loss: 7.296 | train acc: 0.024 | val loss: 7.514 | val acc: 0.018
Epoch 5/5


Validation: 100%|██████████| 129/129 [00:54<00:00,  2.36it/s]


Epoch 5/5 | train loss: 7.279 | train acc: 0.025 | val loss: 7.515 | val acc: 0.017


### Start evaluation with best model

In [31]:
model.load_state_dict(torch.load(os.path.join(logdir, "model_best.pt"), map_location=args['device']))

preds = []
model.eval()
for X, subject_idxs in tqdm(test_loader, desc="Validation"):
    preds.append(model(X.to(args['device'])).detach().cpu())

preds = torch.cat(preds, dim=0).numpy()
np.save(os.path.join(logdir, "submission"), preds)
cprint(f"Submission {preds.shape} saved at {logdir}", "cyan")

Validation: 100%|██████████| 129/129 [00:53<00:00,  2.42it/s]


Submission (16432, 1854) saved at ./output
